In [2]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=3a1c65c64cf9b2df5e7ba80fd554053d2306cd1136a7362ae5bebd3bb5d0d285
  Stored in directory: /Users/akhilakumaripuppala/Library/Caches/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [3]:
import os
import pandas as pd
import numpy as np
from apyori import apriori
from collections import Counter
from datetime import datetime
from itertools import combinations
import matplotlib.pyplot as plt

In [6]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')

orders = orders[orders.eval_set == 'prior']
orders.drop('eval_set', axis = 1, inplace = True)

order_products = pd.read_csv('order_products__prior.csv')

In [29]:
orders_apriori = orders.copy()
orders_by_user = orders.groupby('user_id')['order_number'].max()
products_by_user = orders[['order_id', 'user_id']].merge(
    order_products[['order_id', 'add_to_cart_order']].groupby('order_id').max().rename({'add_to_cart_order': 'order_size'}, axis = 1),
    on = 'order_id')
products_by_user = products_by_user.drop('order_id', axis = 1).groupby('user_id')['order_size'].sum()

In [30]:
orders_apriori.drop(['user_id', 'order_id'], axis = 1, inplace = True)

def order_number_categorical(order_number):
    if order_number in range(3):
        return 'order_number_1-3'
    if order_number in range(3, 5):
        return 'order_number_4-5'
    if order_number in range(5, 10):
        return 'order_number_6-10'
    if order_number in range(10, 20):
        return 'order_number_11-20'
    if order_number in range(20, 40):
        return 'order_number_21-40'
    if order_number in range(40, 60):
        return 'order_number_41-60'
    if order_number >= 60:
        return 'order_number_60+'
    

orders_apriori.order_number = orders_apriori.order_number.map(order_number_categorical)

In [31]:
orders_by_dow = orders.order_dow.value_counts()
products_by_dow = orders[['order_id', 'order_dow']].merge(
    order_products[['order_id', 'add_to_cart_order']].groupby('order_id').max().rename({'add_to_cart_order': 'order_size'}, axis = 1),
    on = 'order_id')
products_by_dow = products_by_dow.drop('order_id', axis = 1).groupby('order_dow')['order_size'].sum()

def dow_categorical(dow):
    if dow in [0, 1]:
        return 'weekend'
    else:
        return 'weekday'
    
orders_apriori.order_dow = orders_apriori.order_dow.map(dow_categorical)

In [32]:
orders_by_hour = orders.order_hour_of_day.value_counts()

products_by_hour = orders[['order_id', 'order_hour_of_day']].merge(
    order_products[['order_id', 'add_to_cart_order']].groupby('order_id').max().rename({'add_to_cart_order': 'order_size'}, axis = 1),
    on = 'order_id')
products_by_hour = products_by_hour.drop('order_id', axis = 1).groupby('order_hour_of_day')['order_size'].sum()

def hour_categorical(hour):
    if hour in range(7):
        return 'early_hours'
    if hour in range(7, 10):
        return 'hour_' + str(hour)
    if hour in range(10, 17):
        return 'peak_hours'
    if hour in range(17, 24):
        return 'hour_' + str(hour)
    
orders_apriori.order_hour_of_day = orders_apriori.order_hour_of_day.map(hour_categorical)

In [33]:
def interval_categorical(interval):
    if np.isnan(interval):
        return 'first_order'
    elif interval in [7, 14, 21]:
        return 'interval_weekly'
    elif interval == 30:
        return 'interval_30+'
    else:
        return 'interval_others'
    
orders_apriori.days_since_prior_order = orders_apriori.days_since_prior_order.map(interval_categorical)

In [34]:
orders_apriori

,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,order_number_1-3,weekday,hour_8,first_order
1,order_number_1-3,weekday,hour_7,interval_others
2,order_number_4-5,weekday,peak_hours,interval_weekly
3,order_number_4-5,weekday,hour_7,interval_others
4,order_number_6-10,weekday,peak_hours,interval_others
...,...,...,...,...
3421077,order_number_6-10,weekday,peak_hours,interval_others
3421078,order_number_11-20,weekday,hour_18,interval_others
3421079,order_number_11-20,weekday,peak_hours,interval_30+
3421080,order_number_11-20,weekend,peak_hours,interval_others


In [35]:
products_id_to_name = {k: v for k, v in zip(products.product_id, products.product_name)}
order_products_names = order_products.copy()
order_products_names['product_name'] = order_products_names.product_id.map(lambda x: products_id_to_name[x])
order_products_names

reorder_proportion = pd.crosstab(order_products_names.product_name, order_products_names.reordered)

reorder_proportion.sort_values(by = 0, ascending = False)


#The two top first-purchases are bananas, following by other produce. 
#At the bottom, some products were purchased only once by some single user.

reordered,0,1
product_name,,
Banana,73956,398609
Bag of Organic Bananas,63537,315913
Organic Strawberries,58838,205845
Organic Baby Spinach,55037,186884
Large Lemon,46402,106255
...,...,...
Brut Prosecco,1,0
Smooth & Silky Head and Shoulders Smooth & Silky 2-in-1 Dandruff Shampoo + Conditioner 13.5 Fl Oz Female Hair Care,1,0
Gluten Free Pantry Sugar Cookie Mix,1,1


In [36]:
reorder_proportion.sort_values(by = 1, ascending = False)

#The ordered champions are also bananas, followed again by strawberries and spinach. 
#People really tend to enjoy them after first try. At the bottom, some products were never reordered.

reordered,0,1
product_name,,
Banana,73956,398609
Bag of Organic Bananas,63537,315913
Organic Strawberries,58838,205845
Organic Baby Spinach,55037,186884
Organic Hass Avocado,43453,170131
...,...,...
Decorating Green Cake Icing,21,0
Lemon Ice Tea Liquid,4,0
"Decorating Gel, Red",7,0


In [37]:
reorder_proportion['total'] = reorder_proportion.sum(axis = 1)
reorder_proportion['0.perc'] = reorder_proportion[0]/reorder_proportion['total']
reorder_proportion['1.perc'] = reorder_proportion[1]/reorder_proportion['total']

reorder_proportion.sort_values(by = ['0.perc', 'total'], ascending = False)[['0.perc', 'total']]

#The top results mean that, all times these products were ordered, it was a first-purchase (since the value is equal to 100%). 
#Since the number of purchases is relatively high, it seems people do tend to dislike them after first try.

#At the bottom, we see those products that tend not to be first-purchases, meaning that users tend to like them after first try.

reordered,0.perc,total
product_name,,
Vicks DayQuil and NyQuil Cold & Flu Relief Combo,1.000000,88
Organic Castor Oil Skin Care,1.000000,67
Organic Whole Rosemary,1.000000,63
Lemon Verbena Scent Glass Cleaner,1.000000,53
Organic Lemongrass,1.000000,52
...,...,...
Soy Powder Infant Formula,0.085714,35
Chocolate Love Bar,0.079208,101
Orange Energy Shots,0.076923,13


In [38]:
reorder_proportion.sort_values(by = ['1.perc', 'total'], ascending = False)[['1.perc', 'total']]


#As expected, this table is the complement of the previous, meaning the products in the bottom there are the top here.
#From all times they were purchased, the top 5 products were reorders more than 91% of the time.

#The products in the bottom were purchased only once. From their names, I would conclude that not all eccentricity 
#has a market demand... The retailer would do good not repurchasing these products themselves.

reordered,1.perc,total
product_name,,
Raw Veggie Wrappers,0.941176,68
Serenity Ultimate Extrema Overnight Pads,0.931034,87
Orange Energy Shots,0.923077,13
Chocolate Love Bar,0.920792,101
Soy Powder Infant Formula,0.914286,35
...,...,...
Vitality Chicken Breasts with Flaxseed & Vitamins Dog Treats,0.000000,1
Vitamin D Gummies,0.000000,1
Wicked Good No-Fry Wing Marinade,0.000000,1


In [39]:
reorder_proportion.total.sort_values(ascending = False)

#Produce is the winner, and bananas are in high demand, selling around 4 times more than the third place! 
#On the other hand, no one seems to want or like those gummies...

product_name
Banana                                                    472565
Bag of Organic Bananas                                    379450
Organic Strawberries                                      264683
Organic Baby Spinach                                      241921
Organic Hass Avocado                                      213584
                                                           ...  
Pure Performance Restorative Shampoo Chamomile & Honey         1
Lindor Peppermint White Chocolate Truffles                     1
Dynostix Rawhide Chew With Meat                                1
Max White With Polishing Star Soft Toothbrush                  1
Cajun Sides Dirty Rice                                         1
Name: total, Length: 49677, dtype: int64

In [40]:
products_bought = sorted(order_products.product_id.unique())
print(len(products_bought), len(products))

#11 products were never purchased

49677 49688


In [41]:
products_not_bought = list(products.product_id[~products.product_id.isin(products_bought)])
[products_id_to_name[product] for product in products_not_bought]

['Protein Granola Apple Crisp',
 'Wasabi Cheddar Spreadable Cheese',
 'Unpeeled Apricot Halves in Heavy Syrup',
 'Chocolate Go Bites',
 'Non-Dairy Coconut Seven Layer Bar',
 'Water With Electrolytes',
 'Ultra Sun Blossom Liquid 90 loads Fabric Enhancers',
 'Sweetart Jelly Beans',
 '12 Inch Taper Candle White',
 'Single Barrel Kentucky Straight Bourbon Whiskey',
 'Pure Squeezed  Lemonade']

In [42]:
products_not_registered = list(pd.Series(products_bought)[~pd.Series(products_bought).isin(products.product_id)])
print(len(products_not_registered), products_not_registered)

#None! 

0 []


In [43]:
add_to_cart = pd.crosstab(order_products_names.product_name, order_products_names.add_to_cart_order)
for i in range(1, 6):
    print('ORDER = ', i)
    print(add_to_cart.sort_values(by = i, ascending = False)[i][:5])
    print('\n')
    

#It is not only that people like bananas, it is also that they are the most common product put from first to fifth 
#in their carts. 

#The other top positions are strawberries, spinach and avocado, also reflecting their total purchase amounts, 
#with one curious exception: the third product put first in the cart is milk.

ORDER =  1
product_name
Banana                    110916
Bag of Organic Bananas     78988
Organic Whole Milk         30927
Organic Strawberries       27975
Organic Hass Avocado       24116
Name: 1, dtype: int64


ORDER =  2
product_name
Banana                    80432
Bag of Organic Bananas    64337
Organic Strawberries      29754
Organic Hass Avocado      26517
Organic Baby Spinach      25025
Name: 2, dtype: int64


ORDER =  3
product_name
Banana                    58577
Bag of Organic Bananas    48972
Organic Strawberries      27809
Organic Baby Spinach      24297
Organic Hass Avocado      24212
Name: 3, dtype: int64


ORDER =  4
product_name
Banana                    43653
Bag of Organic Bananas    36955
Organic Strawberries      25242
Organic Baby Spinach      22733
Organic Hass Avocado      21001
Name: 4, dtype: int64


ORDER =  5
product_name
Banana                    33647
Bag of Organic Bananas    28897
Organic Strawberries      22137
Organic Baby Spinach      20308
Organic Has

In [44]:
transactions = []
for i in range(orders_apriori.shape[0]):
    transactions.append([str(orders_apriori.values[i, j]) for j in range(orders_apriori.shape[1])])
    

start = datetime.now()
rules = apriori(transactions, min_support = 0.005, min_confidence = 0.2, min_lift = 2)
results = list(rules)
print('Execution time: ', datetime.now() - start)

Execution time:  0:04:08.751897


In [ ]:
# the rules are of the kind if A|B then B|A, meaning that they relate to the itemset AB.

# The first element in this RelationRecord, labeled items, contains the itemset considered for this rule, meaning AB.

# The second is the support for AB.

# The third contains the rules; first if A then B, second if B then A.

# The first element inside the first rule, labeled items_base, is A. In the second rule, it is B.

# The second element, items_add, is B for the first rule, and A for the second.

# The third and fourth elements are, as it is evident, the confidence and lift.

In [45]:
A = []
B = []
support = []
confidence = []
lift = []

for result in results:
    s = result[1]
    result_rules = result[2]
    for result_rule in result_rules:
        a = list(result_rule[0])
        b = list(result_rule[1])
        c = result_rule[2]
        l = result_rule[3]
        A.append(a)
        B.append(b)
        support.append(s)
        confidence.append(c)
        lift.append(l)
        
rules_df = pd.DataFrame({'A' : A,
                         'B' : B,
                         'support' : support,
                         'confidence' : confidence,
                         'lift' : lift})
rules_df = rules_df.sort_values(by = 'lift', ascending = False).reset_index(drop = True)
len(rules_df)

#38 rules werer generated

38

In [46]:
#The first one is the orders_apriori dataframe designed above, reflecting purchase habits.

rules_df

,A,B,support,confidence,lift
0,"[first_order, peak_hours]","[weekend, order_number_1-3]",0.013600,0.367834,8.057873
1,"[weekend, order_number_1-3]","[first_order, peak_hours]",0.013600,0.297930,8.057873
2,"[first_order, weekend]","[peak_hours, order_number_1-3]",0.013600,0.594952,8.047623
3,"[weekend, order_number_1-3]",[first_order],0.022859,0.500763,7.807082
4,[first_order],"[weekend, order_number_1-3]",0.022859,0.356386,7.807082
5,[first_order],"[peak_hours, weekend, order_number_1-3]",0.013600,0.212032,7.799732
6,"[peak_hours, weekend, order_number_1-3]",[first_order],0.013600,0.500292,7.799732
7,"[peak_hours, order_number_1-3]",[first_order],0.036974,0.500126,7.797151
8,[first_order],"[peak_hours, order_number_1-3]",0.036974,0.576435,7.797151
9,[first_order],"[peak_hours, weekday, order_number_1-3]",0.023374,0.364402,7.795650
